In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels as sm
import os
from pandas.api.types import is_string_dtype, is_numeric_dtype
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [ ]:
def get_cat_columns_by_type(df):
    out = []
    for colname, col_values in df.items():
        if is_string_dtype(col_values):
            out.append((colname,'string') )
        elif not is_numeric_dtype(col_values):
            out.append((colname,'categorical') )
    return out

def get_missing_values_percentage(df):
    missing_values_counts_list = df.isnull().sum()
    total_values = np.product(df.shape)
    total_missing = missing_values_counts_list.sum()
    # percent of data that is missing
    return (total_missing/total_values) * 100

def handle_missing_values(df_in, na_dict=None, inplace=True):
    if(inplace):
        df = df_in
    else:
        df = df_in.copy()
 
    if na_dict is None:
        na_dict = {}

    for colname, col_values in df.items():
        if colname in na_dict:
            if na_dict[colname] == 'drop_rows':
                df.dropna(subset=[colname], inplace=True)
                continue
            elif na_dict[colname] == 'drop_col':
                df.drop(colname, axis=1, inplace=True)
                continue

        if is_numeric_dtype(col_values):
            if pd.isnull(col_values).sum():
                df[colname+'_na'] = pd.isnull(col_values)
                filler = na_dict[colname] if colname in na_dict else col_values.median()
                df[colname] = col_values.fillna(filler)
                na_dict[colname] = filler
    return (df,na_dict)


In [ ]:
LOCAL_DIR = os.path.join("datasets", "iowa_housing")
TRAIN_FILE = os.path.join(LOCAL_DIR,'train.csv')
TEST_FILE = os.path.join(LOCAL_DIR,'test.csv')

In [ ]:
df_raw = pd.read_csv(TRAIN_FILE, low_memory=False)
df_test = pd.read_csv(TEST_FILE, low_memory=False)


In [ ]:
get_cat_columns_by_type(df_raw)

In [ ]:
df = df_raw.copy()

In [ ]:
df_id = df['Id']
test_id = df_test['Id']

In [ ]:
y = df['SalePrice']

In [ ]:
df.drop('Id', axis = 1, inplace = True)
df.drop('SalePrice', axis = 1, inplace = True)
df_test.drop("Id", axis = 1, inplace = True)

In [ ]:
ntrain = df.shape[0]
ntest = df_test.shape[0]

In [ ]:
combined = pd.concat((df, df_test),sort=False).reset_index(drop=True)

In [ ]:
print('Train shape: {}, Test shape: {}, Combined shape: {}'.format(df.shape,df_test.shape,combined.shape))

In [ ]:
combined.describe()

In [ ]:
nans = combined.isna().sum().sort_values(ascending=False)
nans

In [ ]:
combined['MSSubClass'].value_counts()

In [ ]:
combined['MSSubClass'] = combined['MSSubClass'].apply(str)

In [ ]:
combined['YrSold'] = combined['YrSold'].astype(str)

In [ ]:
combined['MoSold'] = combined['MoSold'].astype(str)

### fill nans with some specific values

In [ ]:
for col in ['Exterior1st','Exterior2nd','SaleType','Functional','Electrical','KitchenQual']:
    combined[col] = combined[col].fillna(combined[col].mode()[0])

In [ ]:
for col in ['GarageYrBlt', 'GarageArea', 'GarageCars']:
    combined[col] = combined[col].fillna(0)

In [ ]:
for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    combined[col] = combined[col].fillna('None')

In [ ]:
for col in ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2','PoolQC']:
    combined[col] = combined[col].fillna('None')

In [ ]:
combined,_ = handle_missing_values(combined,inplace=False)

In [ ]:
# add some relevant extra columns
combined['Total_porch_SF'] = (combined['OpenPorchSF'] + combined['3SsnPorch'] +
                              combined['EnclosedPorch'] + combined['ScreenPorch'] +
                              combined['WoodDeckSF'])


In [ ]:
combined['Total_SF'] = combined['TotalBsmtSF'] + combined['1stFlrSF'] + combined['2ndFlrSF']

In [ ]:
combined['Total_SF_2'] = (combined['BsmtFinSF1'] + combined['BsmtFinSF2'] +
                       combined['1stFlrSF'] + combined['2ndFlrSF'])

In [ ]:
combined['Total_Bathrooms'] = (combined['FullBath'] + (0.5 * combined['HalfBath']) +
                               combined['BsmtFullBath'] + (0.5 * combined['BsmtHalfBath']))

In [ ]:
combined['haspool'] = combined['PoolArea'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
combined['has2ndfloor'] = combined['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
combined['hasgarage'] = combined['GarageArea'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
combined['hasbsmt'] = combined['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
combined['hasfireplace'] = combined['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
combined = pd.get_dummies(combined).reset_index(drop=True)

In [ ]:
print('Percentage of missing values: {}'.format(get_missing_values_percentage(combined)))    

In [ ]:
combined.columns

In [ ]:
combined.describe()